In [ ]:
from DemoServerDBTWO import ConnectDatabase

In [ ]:
temp = ConnectDatabase()

In [ ]:
temp

In [ ]:
temp.cur.execute('SELECT * from transaction_master')

In [ ]:
trans_master = temp.cur.fetchall()

In [ ]:
trans_master[0]

In [ ]:
col_names = [desc[0] for desc in temp.cur.description]

In [ ]:
col_names

In [ ]:
import yaml
with open('/home/ubuntu/MappingBuffer2.yml', 'r') as f:
    doc = yaml.load(f)

In [ ]:
doc['column_map']['TRANSACTION_MASTER']

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
master_df = pd.DataFrame(trans_master)

In [ ]:
master_df.head(6)

In [ ]:
master_df.columns = col_names

In [ ]:
master_df.head(6)

In [ ]:
master_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id']:'cust_id'}, inplace = True)
master_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['revenue']:'revenue'}, inplace = True)
master_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['product_id']:'product_id'}, inplace = True)
master_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['timestamp']:'timestamp'}, inplace = True)

In [ ]:
master_df.head(6)

In [ ]:
cust_prod = master_df[['product_id','cust_id']]

In [ ]:
cust_prod.head(6)

In [ ]:
cust_prod.shape

In [ ]:
values = pd.DataFrame([1]*len(cust_prod))

In [ ]:
values.rename(columns={0:'value'}, inplace=True)

In [ ]:
cust_prod_df = pd.concat([cust_prod,values],axis=1)

In [ ]:
cust_prod_df.head(6)

In [ ]:
cust_prod_df['product_id'] = cust_prod_df['product_id'].astype('category')

In [ ]:
cust_prod_df['product_id'] = cust_prod_df['product_id'].cat.codes

In [ ]:
cust_prod_df.head(6)

In [ ]:
cust_prod_df.to_csv('/home/ubuntu/cust_prod')

In [1]:
import os
import sys

spark_path = "/opt/spark/"

os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.9-src.zip")

from pyspark import SparkContext
from pyspark import SparkConf

sc = SparkContext("local", "test")

In [2]:
sc

In [3]:
data = sc.textFile('/home/ubuntu/client3_cust_prod')

In [4]:
data.take(5)

[u',product_id,cust_id,rating',
 u'0,3249,17850,1',
 u'1,2649,17850,1',
 u'2,2855,17850,1',
 u'3,2803,17850,1']

In [5]:
data = data.zipWithIndex().filter(lambda tup: tup[1] > 0).map(lambda tup: tup[0])

In [6]:
data.first()

u'0,3249,17850,1'

In [7]:
data.count()

406829

In [8]:
clean_data = data.map(lambda x:x.split(','))

In [9]:
clean_data.take(5)

[[u'0', u'3249', u'17850', u'1'],
 [u'1', u'2649', u'17850', u'1'],
 [u'2', u'2855', u'17850', u'1'],
 [u'3', u'2803', u'17850', u'1'],
 [u'4', u'2802', u'17850', u'1']]

In [10]:
rating = clean_data.map(lambda y: int(y[3]))

In [11]:
rating.take(5)

[1, 1, 1, 1, 1]

In [12]:
users = clean_data.map(lambda y : int(y[2]))

In [13]:
users_unique = users.distinct()

In [14]:
users_unique.count()

4372

In [15]:
prod = clean_data.map(lambda y : int(y[1]))

In [16]:
prod_unique = prod.distinct()

In [17]:
prod_unique.count()

3684

In [18]:
from pyspark.mllib.recommendation import ALS,MatrixFactorizationModel,Rating

In [19]:
mls = data.map(lambda l: l.split(','))

In [20]:
mls.take(5)

[[u'0', u'3249', u'17850', u'1'],
 [u'1', u'2649', u'17850', u'1'],
 [u'2', u'2855', u'17850', u'1'],
 [u'3', u'2803', u'17850', u'1'],
 [u'4', u'2802', u'17850', u'1']]

In [21]:
ratings = mls.map(lambda x: Rating(int(x[2]), int(x[1]), float(x[3])))

In [22]:
print ratings.take(5)

[Rating(user=17850, product=3249, rating=1.0), Rating(user=17850, product=2649, rating=1.0), Rating(user=17850, product=2855, rating=1.0), Rating(user=17850, product=2803, rating=1.0), Rating(user=17850, product=2802, rating=1.0)]


In [23]:
train, test = ratings.randomSplit([0.7,0.3],7856)

In [24]:
train.count(), test.count()

(284691, 122138)

In [25]:
train.cache()

PythonRDD[21] at RDD at PythonRDD.scala:48

In [26]:
test.cache()

PythonRDD[22] at RDD at PythonRDD.scala:48

### ALS MODEL ###

In [28]:
rank = 5
numIterations = 10

In [29]:
model = ALS.train(train, rank, numIterations)

In [30]:
model.productFeatures().first()

(0,
 array('d', [0.2059554010629654, -0.13989338278770447, -0.5379576086997986, -0.015764115378260612, -0.3808843195438385]))

In [31]:
model.userFeatures().first()

(12346,
 array('d', [0.40648752450942993, -0.27611497044563293, -1.0616824626922607, -0.031083371490240097, -0.751642107963562]))

In [32]:
print model.recommendUsers(3249,10)

[Rating(user=16446, product=3249, rating=0.9933975365376676), Rating(user=15361, product=3249, rating=0.9826976498929281), Rating(user=15512, product=3249, rating=0.9825822319852507), Rating(user=13651, product=3249, rating=0.9803553557267227), Rating(user=13405, product=3249, rating=0.9803076158810162), Rating(user=13140, product=3249, rating=0.980299908924447), Rating(user=14506, product=3249, rating=0.9802997085161186), Rating(user=14882, product=3249, rating=0.9802922489270838), Rating(user=14030, product=3249, rating=0.9802902473170133), Rating(user=16737, product=3249, rating=0.9802877502448843)]


In [33]:
print model.recommendProducts(17850, 10)

[Rating(user=17850, product=1512, rating=0.9818333760067696), Rating(user=17850, product=99, rating=0.9815311177579562), Rating(user=17850, product=2848, rating=0.9814517253538177), Rating(user=17850, product=29, rating=0.9814015532266696), Rating(user=17850, product=1005, rating=0.980640200625386), Rating(user=17850, product=30, rating=0.9805654000254741), Rating(user=17850, product=31, rating=0.9804539941905492), Rating(user=17850, product=2631, rating=0.9803512042551328), Rating(user=17850, product=76, rating=0.9803166104852833), Rating(user=17850, product=676, rating=0.9803101056458157)]


In [34]:
print model.predict(17850,3249)

0.980264809765


In [ ]:
### model.recommendProductsForUsers(1)

In [35]:
pred_input = train.map(lambda x:(x[0], x[1]))

In [36]:
pred = model.predictAll(pred_input)

In [37]:
pred.take(5)

[Rating(user=17231, product=1084, rating=0.9802653425691302),
 Rating(user=17231, product=1084, rating=0.9802653425691302),
 Rating(user=17231, product=1053, rating=0.9802646162042457),
 Rating(user=17231, product=1053, rating=0.9802646162042457),
 Rating(user=17231, product=3517, rating=0.9802660890053336)]

In [38]:
true_reorg = train.map(lambda x:((x[0],x[1]), x[2]))

In [39]:
true_reorg.take(5)

[((17850, 3249), 1.0),
 ((17850, 2649), 1.0),
 ((17850, 2855), 1.0),
 ((17850, 2803), 1.0),
 ((17850, 1605), 1.0)]

In [40]:
pred_reorg = pred.map(lambda x:((x[0],x[1]), x[2]))

In [41]:
pred_reorg.take(5)

[((17231, 1084), 0.9802653425691302),
 ((17231, 1084), 0.9802653425691302),
 ((17231, 1053), 0.9802646162042457),
 ((17231, 1053), 0.9802646162042457),
 ((17231, 3517), 0.9802660890053336)]

In [42]:
true_pred = true_reorg.join(pred_reorg)

In [43]:
true_pred.take(5)

[((14257, 165), (1.0, 0.9802647902736983)),
 ((18079, 1797), (1.0, 0.9802656619774006)),
 ((13001, 3137), (1.0, 0.9802642296775229)),
 ((13001, 3137), (1.0, 0.9802642296775229)),
 ((13001, 3137), (1.0, 0.9802642296775229))]

### EVALUATION 

In [44]:
from math import sqrt

In [45]:
MSE = true_pred.map(lambda r:(r[1][0] - r[1][1])**2).mean()

In [46]:
RMSE = sqrt(MSE)

In [47]:
print RMSE

0.019734848452


In [48]:
test_input = test.map(lambda x:(x[0],x[1]))

In [49]:
pred_test = model.predictAll(test_input)

In [50]:
test_reorg = test.map(lambda x:((x[0],x[1]), x[2]))

In [51]:
pred_reorg = pred_test.map(lambda x:((x[0],x[1]), x[2]))

In [52]:
test_pred = test_reorg.join(pred_reorg)

In [53]:
test_MSE = test_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()

In [54]:
test_RMSE = sqrt(test_MSE)

In [55]:
print test_RMSE

0.0198727775014
